In [ ]:
!pip install langchain langchain-community langchain-chroma langchain-ollama langchain-huggingface sentence-transformers transformers pypdf

  Using cached langchain_community-0.3.16-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_chroma-0.2.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_ollama-0.2.3-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached pypdf-5.2.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.115.8-py3-none-any.whl.metadata (27 kB)
  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached uvicorn-0.34.0-py3-none-any.whl.m

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
CSV_PATH = "./gdrive/MyDrive/Agroknow/Incidents"
PDF_PATH = "./gdrive/MyDrive/Agroknow/PDFs"
CHROMA_PATH = "./gdrive/MyDrive/Agroknow/chroma"
FAISS_INDEX_PATH = "./faiss"

In [ ]:
import os
from tqdm.notebook import tqdm
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader, PyPDFLoader, UnstructuredPDFLoader
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings


def load_and_chunk_csv(CSV_PATH):
    """
    Load and process CSV files to create Document objects for use with LangChain using Pandas.

    Args:
        CSV_PATH (str): Path to the directory containing CSV files.

    Returns:
        list[Document]: A list of processed Document objects.
    """
    print("🔍 Loading and chunking CSV data with Pandas...")
    chunks = []

    # Check if CSV_PATH is a directory
    if not os.path.isdir(CSV_PATH):
        print(f"Error: {CSV_PATH} is not a directory")
        return chunks

    # Process each CSV file in the directory
    for filename in os.listdir(CSV_PATH):
        if filename.endswith('.csv'):
            file_path = os.path.join(CSV_PATH, filename)
            try:
                # Read the CSV into a Pandas DataFrame
                df = pd.read_csv(file_path, sep='\t')
                df['summary'] = df['summary'].fillna(df['description'])

                # Process each row in the DataFrame
                for _, row in df.iterrows():
                    # Extract relevant information
                    title = row.get('title', '')
                    date = row.get('date', '')
                    hazard = row.get('hazard', '')
                    summary = row.get('summary', '')
                    product = row.get('product', '')
                    origin = row.get('origin', '')
                    year = row.get('date', '').split('-')[0] if pd.notna(row.get('date', '')) else ''  # Extract year from date

                    # Create a concise content string
                    content = (
                        f"Title: {title}\n"
                        f"Date: {date}\n"
                        f"Hazard: {hazard}\n"
                        f"Product: {product}\n"
                        f"Origin: {origin}\n"
                        f"Year: {year}\n"
                        f"Summary: {summary}"
                    )

                    # Create a Document object
                    chunk = Document(
                        page_content=content,
                        metadata={
                            "title": title,
                            "date": date,
                            "hazard": hazard,
                            "product": product,
                            "origin": origin,
                            "year": year,
                            "source_file": filename  # Optional: Track source file
                        }
                    )
                    chunks.append(chunk)

            except Exception as e:
                print(f"An unexpected error occurred with {filename}: {str(e)}")

    print(f"✂️ Loaded and processed {len(chunks)} chunks from CSV files using Pandas")
    return chunks


def load_and_split_pdfs(PDF_PATH):
    """Load and split PDFs from a directory with a progress bar."""
    print("🔍 Loading PDF documents...")

    if not os.path.isdir(PDF_PATH):
        print(f"❌ Error: {PDF_PATH} is not a directory")
        return []

    # Load all PDFs at once
    loader = PyPDFDirectoryLoader(PDF_PATH)
    documents = loader.load()
    print(f"📚 Loaded {len(documents)} PDF documents")

    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)

    # Split documents with progress bar
    chunks = []
    for doc in tqdm(documents, total=len(documents), desc="🔄 Splitting PDFs"):
        chunks.extend(text_splitter.split_documents([doc]))

    print(f"✅ Finished splitting PDFs into {len(chunks)} chunks")
    return chunks


def calculate_chunk_ids(chunks, prefix):
    # Assign unique IDs to each chunk based on its source and index
    for index, chunk in enumerate(chunks):
        if prefix == "pdf": #pdf
            source = chunk.metadata.get("source")
            page = chunk.metadata.get("page")
            chunk_id = f"pdf:{source}:{page}:{index}"
        else:  # CSV
            filename = chunk.metadata.get("source_file", "unknown")
            chunk_id = f"csv:{filename}:{index}"

        chunk.metadata["id"] = chunk_id

    return chunks


def add_to_chroma(chunks: list[Document], embedding_model):
     # Add chunks to Chroma database
    print("💾 Adding to Chroma...")
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding_model
    )

    # Get existing items in the database
    existing_items = db.get(include=[])
    existing_ids = set(existing_items["ids"])
    print(f"📊 Number of existing documents in DB: {len(existing_ids)}")

    # Filter out chunks that already exist in the database
    new_chunks = [chunk for chunk in chunks if chunk.metadata["id"] not in existing_ids]

    if new_chunks:
        # Add new chunks to the database
        print(f"👉 Adding {len(new_chunks)} new documents")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]

        # Use tqdm to show progress
        pbar = tqdm(total=len(new_chunks), desc="Adding documents", unit="doc")

        for chunk, chunk_id in zip(new_chunks, new_chunk_ids):
            db.add_documents([chunk], ids=[chunk_id])
            pbar.update(1)

        pbar.close()

        print("✅ Database updated and persisted")
    else:
        print("✅ No new documents to add")


def reset_chroma_database(embedding_model):
    # Print a message indicating the start of the database reset process
    print("🗑️ Clearing entire database")

    # Initialize a new Chroma database instance
    # This connects to the existing database using the specified persistence directory and embedding model
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding_model
    )

    # Delete the entire collection in the database
    # This removes all documents and their associated embeddings
    db.delete_collection()
    print("✅ Database reset completed")


def add_to_faiss(chunks: list[Document], embedding_model):
    """
    Add chunks to a FAISS vectorstore, ensuring no duplicates are added.

    Args:
        chunks (list[Document]): A list of Document objects to add.
        embedding_model (SentenceTransformerEmbeddings): Embedding model for generating embeddings.
    """
    print("💾 Adding to FAISS...")

    # Filter valid documents (ensure non-empty content)
    valid_chunks = [chunk for chunk in chunks if chunk.page_content.strip()]
    if not valid_chunks:
        print("⚠️ No valid documents to process. Aborting.")
        return

    # Check if FAISS index exists
    if os.path.exists(FAISS_INDEX_PATH):
        print("📂 Loading existing FAISS index...")
        vectorstore = FAISS.load_local(FAISS_INDEX_PATH, embedding_model)
    else:
        print("🆕 Creating a new FAISS index...")

        # Generate embeddings for the valid documents
        texts = [chunk.page_content for chunk in valid_chunks]
        metadatas = [chunk.metadata for chunk in valid_chunks]
        embeddings = embedding_model.embed_documents(texts)

        # Ensure embeddings are generated correctly
        if not embeddings or len(embeddings) != len(texts):
            print("⚠️ Embedding generation failed. Aborting.")
            return

        # Initialize FAISS with the valid documents
        vectorstore = FAISS.from_texts(texts, embedding_model, metadatas=metadatas)
        vectorstore.save_local(FAISS_INDEX_PATH)
        print("✅ New FAISS index created and saved.")
        return  # No need to continue; the index has just been created.

    # Extract existing document IDs
    existing_ids = set()
    for doc in vectorstore.docstore._dict.values():
        if "id" in doc["metadata"]:
            existing_ids.add(doc["metadata"]["id"])
    print(f"📊 Number of existing documents in DB: {len(existing_ids)}")

    # Filter out chunks that already exist
    new_chunks = [chunk for chunk in valid_chunks if chunk.metadata["id"] not in existing_ids]

    if new_chunks:
        print(f"👉 Adding {len(new_chunks)} new documents")

        # Extract text and metadata for new documents
        new_texts = [chunk.page_content for chunk in new_chunks]
        new_metadatas = [chunk.metadata for chunk in new_chunks]

        # Generate embeddings for the new texts
        embeddings = embedding_model.embed_documents(new_texts)
        if not embeddings or len(embeddings) != len(new_texts):
            print("⚠️ Embedding generation failed. Aborting.")
            return

        # Add new documents to FAISS
        vectorstore.add_texts(new_texts, metadatas=new_metadatas, embeddings=embeddings)

        # Save the updated FAISS index
        vectorstore.save_local(FAISS_INDEX_PATH)
        print("✅ Database updated and saved")
    else:
        print("✅ No new documents to add")


In [ ]:
csv_chunks = load_and_chunk_csv(CSV_PATH)
csv_chunks = calculate_chunk_ids(csv_chunks, "csv")

🔍 Loading and chunking CSV data with Pandas...
✂️ Loaded and processed 5434 chunks from CSV files using Pandas


In [ ]:
pdf_chunks = load_and_split_pdfs(PDF_PATH)
pdf_chunks = calculate_chunk_ids(pdf_chunks, "pdf")

🔍 Loading PDF documents...
📚 Loaded 6452 PDF documents


🔄 Splitting PDFs:   0%|          | 0/6452 [00:00<?, ?it/s]

✅ Finished splitting PDFs into 39370 chunks


In [ ]:
# Combine all chunks
all_chunks = pdf_chunks + csv_chunks
print(f"📚 Total chunks: {len(all_chunks)}")

📚 Total chunks: 44804


In [ ]:
print(csv_chunks[5000])

page_content='Title: Listeria monocytogenes in enoki mushrooms by Xin Ao International Group Corp. from United States
Date: 2023-03-24
Hazard: listeria monocytogenes
Product: enoki mushrooms
Origin: united states
Year: 2023
Summary: Xin Ao International Group Corp, 601 E. Linden Ave Linden NJ 07036is initiating a recall of Enoki Mushroom, 200g, Plastic Bag, 25 bags per case because the product may be contaminated with Listeria monocytogenes.' metadata={'title': 'Listeria monocytogenes in enoki mushrooms by Xin Ao International Group Corp. from United States', 'date': '2023-03-24', 'hazard': 'listeria monocytogenes', 'product': 'enoki mushrooms', 'origin': 'united states', 'year': '2023', 'source_file': 'case_13_listeria_in_enoki_mushrooms_2023.csv', 'id': 'csv:case_13_listeria_in_enoki_mushrooms_2023.csv:5000'}


In [ ]:
print(pdf_chunks[5000])

page_content='1 3 / 1 3 
The Government of Canada is committed to food safety. The Public Health Agency of
Canada is leading the human health investigation of this outbreak and is in regular
contact with its federal and provincial partners to monitor and take collaborative steps to
address the outbreak.
Health Canada provides food-related health risk assessments to determine if the
presence of a certain substance or microorganism poses a health risk to consumers.' metadata={'source': 'gdrive/MyDrive/Agroknow/PDFs/case 09/canada.ca-Public Health Notice Update  Outbreak of Listeria infections linked to packaged salad products produc.pdf', 'page': 12, 'page_label': '13', 'id': 'pdf:gdrive/MyDrive/Agroknow/PDFs/case 09/canada.ca-Public Health Notice Update  Outbreak of Listeria infections linked to packaged salad products produc.pdf:12:5000'}


In [ ]:
# Load SentenceTransformer model
embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

<ipython-input-3-0cbf9acebbb5>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access publi

In [ ]:
add_to_chroma(all_chunks, embedding_model)

💾 Adding to Chroma...
📊 Number of existing documents in DB: 19466
👉 Adding 25338 new documents


Adding documents:   0%|          | 0/25338 [00:00<?, ?doc/s]

✅ Database updated and persisted


In [ ]:
# reset_chroma_database(embedding_model)

🗑️ Clearing entire database
✅ Database reset completed


In [ ]:
# add_to_faiss(csv_chunks, embedding_model)

💾 Adding to FAISS...
🆕 Creating a new FAISS index...
✅ New FAISS index created and saved.


In [ ]:
PROMPT_TEMPLATE = """
Answer the question based ONLY on the following context:

Context:
{context}

Instructions:
You are an assistant specializing in food safety and root cause analysis. Your expertise includes contamination incidents, recalls, and outbreaks.

Question: {question}
Answer:
"""

In [ ]:
def query_rag(query_text, retriever, llm):
    """
    Query the Chroma database with a question using RetrievalQA.

    Args:
        query_text (str): The question to query.
        retriever: The retriever instance for Chroma.
        llm: The language model to use for generating responses.

    Returns:
        tuple: A tuple containing the response text and formatted sources.
    """
    print("🔍 Querying the Chroma database...")

    # Create a prompt template
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template=PROMPT_TEMPLATE
    )

    # Set up the QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_template}
    )

    # Execute the query
    result = qa_chain.invoke({"query": query_text})
    response_text = result['result']
    source_documents = result['source_documents']

    print(source_documents)

    # Format sources
    sources = "\n".join([doc.metadata.get("id", "Unknown Source") for doc in source_documents])
    return response_text, sources

In [ ]:
from transformers import pipeline

# Create a pipeline
pipe = pipeline(
    task="text-generation",
    model="Qwen/Qwen2.5-1.5B-Instruct"
    return_full_text=False,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.2,
)

# Wrap the Hugging Face pipeline with LangChain
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [ ]:
# Initialize Chroma database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_model)

# Initialize the retriever
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 20, "fetch_k": 50})

In [ ]:
queries = [
    # Case #1: Lead Poisoning in Applesauce Products
    "What specific products were involved in the Lead Poisoning in Applesauce Products incidents?",
    "For the Lead Poisoning in Applesauce Products, was the cinnamon intentionally adulterated? / Was this economically motivated adulteration?",
    # Case #2: Salmonella spp. in Cantaloupe Melons, 2023
    "For the Salmonella spp. in Cantaloupe Melons, 2023 incidents, had there been any similar incidents with the same product or facility/brand in the past?",
    "For the Salmonella spp. in Cantaloupe Melons, 2023 incidents, what was the origin of the pathogen that caused the outbreak?",
    # Case #3: E. Coli in Raw Milk Cheese
    "For the E. Coli in Raw Milk Cheese incidents, what was the scale of this outbreak? (number of affected manufacturing plants, geographical spread, people hospitalized. etc.)",
    "For the E. Coli in Raw Milk Cheese incidents, ss the problem (as in Root Cause) traced back to raw milk quality or absence of good manufacturing practices during production?",
    # Case #4: Listeria in Fresh Peaches, Plums & Nectarines
    "For the Listeria in Fresh Peaches, Plums & Nectarines incidents, why were the fruits recalled since they were past their expiration date?",
    "For the Listeria in Fresh Peaches, Plums & Nectarines incidents, how was the Listeria contamination initially detected?",
    # Case #5: Norovirus in Oysters
    "For the Norovirus in Oysters incidents, were the oysters processed post harvest?",
    "For the Norovirus in Oysters incidents, were there any recent environmental changes or events (for example heavy rain)that could have contributed to the initial contamination of the oysters?",
    # Case #6: Salmonella in Fresh Diced Onions
    "For the Salmonella in Fresh Diced Onions incidents, were there any similar incidents in the past involving the same supplier or distributor? If yes, were these linked to diced vegetables?",
    "For the Salmonella in Fresh Diced Onions incidents, were the findings concerning the root cause analysis conflicting?",
    # Case #7: Salmonella in Alfalfa Sprouts
    "For the Salmonella in Alfalfa Sprouts incidents, how was the contamination initially detected?",
    "For the Salmonella in Alfalfa Sprouts incidents, were there findings of poor food safety and hygiene practices in the manufacturing facilities?",
    # Case #8: Salmonella in Raw Cookie Dough
    "For the Salmonella in Raw Cookie Dough incidents, what ingredients were used in this product and which one/ones were the source of contamination?",
    "For the Salmonella in Raw Cookie Dough incidents, were all food safety and hygiene practices met during production handling and packaging of the cookie dough?",
    # Case #9: Listeria in Leafy Greens
    "For the Listeria in Leafy Greens incidents, was the strain identified either in the harvesting equipment, the manufacturing facilities or both?",
    "For the Listeria in Leafy Greens incidents, what prompted the FDA investigation?",
    # Case #10: Salmonella in Ground Beef
    "For the Salmonella in Ground Beef incidents, what is the most likely cause of contamination?",
    "What was the scale of the Salmonella in Ground Beef outbreak?",
    # Case #11: Listeria in Deli Meat and Cheese, 2023
    "For the Listeria in Deli Meat and Cheese, 2023 incidents, what is the most likely cause of contamination?",
    "What was the scale of the Listeria in Deli Meat and Cheese, 2023 outbreak?",
    # Case #12: Salmonella in Flour
    "For the Salmonella in Flour incidents, what is the most likely cause of contamination?",
    "What was the scale of the Salmonella in Flour outbreak?",
    # Case #13: Listeria in Enoki Mushrooms
    "For the Listeria in Enoki Mushrooms incidents, are enoki mushrooms subject to processing (heat treatment) by the consumer before consumption or are they consumed raw?",
    "For the Listeria in Enoki Mushrooms incidents, was the manufacturer involved in similar incidents in the past?",
    # Case #14: Salmonella in Cocoa
    "For the Salmonella in Cocoa incidents, what is the most likely cause of contamination?",
    "What was the scale of the Salmonella in Cocoa outbreak?",
    # Case #15: Cronobacter Sakazakii in Infant Formulae Power
    "What prompted the Cronobacter Sakazakii in Infant Formulae Power investigation?",
    "Was the Cronobacter Sakazakii in Infant Formulae Power outbreak linked to poor manufacturing practices?",
]

In [ ]:
query_text="What specific products were involved in the Lead Poisoning in Applesauce Products incidents?"

response, sources = query_rag(query_text, retriever, llm)

🔍 Querying the Chroma database...


KeyboardInterrupt: 

In [ ]:
for query_text in queries:
    response, sources = query_rag(query_text, retriever, llm)
    print("########################################")
    print("==========Query==========")
    print(query_text)
    print("==========Response==========")
    print(response)
    print("==========Sources==========")
    print(sources)
    print()

🔍 Querying the Chroma database...
[Document(id='pdf:gdrive/MyDrive/Agroknow/PDFs/extra/09.07/politico.com-Applesauce pouches linked to lead poisoning may have been contaminated on purpose FDA foods chief say.pdf:1:27014', metadata={'id': 'pdf:gdrive/MyDrive/Agroknow/PDFs/extra/09.07/politico.com-Applesauce pouches linked to lead poisoning may have been contaminated on purpose FDA foods chief say.pdf:1:27014', 'page': 1, 'page_label': '2', 'source': 'gdrive/MyDrive/Agroknow/PDFs/extra/09.07/politico.com-Applesauce pouches linked to lead poisoning may have been contaminated on purpose FDA foods chief say.pdf'}, page_content='with state and local health departments to find all the cases of lead poisoning linked to the\ncontaminated applesauce products.\nThe FDA foods chief added that the agency intends to track down any perpetrator.\n“We’re going to chase that data and find whoever was responsible and hold them\naccountable,” Jones said.'), Document(id='pdf:gdrive/MyDrive/Agroknow/PDFs/ca

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[Document(id='pdf:gdrive/MyDrive/Agroknow/PDFs/case 05/ilfattoalimentare.it-maxi-stop to production in France Recalls in Italy.pdf:0:10929', metadata={'id': 'pdf:gdrive/MyDrive/Agroknow/PDFs/case 05/ilfattoalimentare.it-maxi-stop to production in France Recalls in Italy.pdf:0:10929', 'page': 0, 'page_label': '1', 'source': 'gdrive/MyDrive/Agroknow/PDFs/case 05/ilfattoalimentare.it-maxi-stop to production in France Recalls in Italy.pdf'}, page_content='according to estimates by the regional shellfish farming committee (CRC) questioned by\nAFP .\nAs reported by Le Monde , according to the prefect of Morbihan, the norovirus\ncontamination of oysters was most likely caused by a series of coinciding factors: the\ngastroenteritis epidemic that spread during the holiday season, favored by low\ntemperatures, and the heavy and intense rains that hit France. This may have caused\noverloads of purification plants and overflows in rivers, which thus poured the water'), Document(id='pdf:gdrive/MyDr